# Gemini Data Extraction from In-Store Images

### Imports and Setup

This section handles the initial setup of the environment. It imports the necessary libraries for interacting with the Gemini API and Google Cloud services. It also initializes the Gemini client with the project and location details, which is a prerequisite for making any API calls.


In [1]:
from google import genai
from google.genai.types import HttpOptions, Part, GenerationConfig
import json

In [2]:
import os

PROJECT_ID = "sandbox-401718"  # @param {type:"string"}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

### System Instructions, Prompt, and Response Schema

This section defines the core instructions for the Gemini model. The `system_instructions` variable sets the model's persona as a JSON extraction engine, while the `prompt` variable provides detailed rules for extracting product information from the images. This includes the desired JSON schema for the output, ensuring that the model returns data in a structured and predictable format.

In [3]:
system_instructions = "You are a highly reliable JSON extraction engine. Your task is to analyze the provided image of a product shelf display and extract structured product data for all **identifiable products**."

In [4]:
prompt = f"""
### Extraction Rules:

1.  **Product Identification:** Identify all distinct products on the shelf. A product is typically defined by its unique packaging and corresponding electronic shelf label (ESL).
2.  **Extractable Fields:** For each product, focus on the following properties: `UPC`, `Brand`, `Description`, `Price`, `Size`, and `UnitOfMeasure`.
3.  **Field-Specific Instructions:**
    *   **`UPC`**: Locate the UPC on the electronic shelf label (ESL) positioned directly beneath the product. If the UPC is not clearly visible or is in a non-standard format, try to find it using the google_search tool with the product's brand, description and size if appropriate. Do not guess the UPC; if you cannot find it with high confidence, leave the field blank.

        **UPC Format Rules:**

        UPC codes have a standard format, most commonly the 12-digit UPC-A version used in North America. A UPC (Universal Product Code) is the number, while the barcode is the machine-readable visual representation of that number. Both are governed by the global standards organization GS1.

        *   **The standard 12-digit format (UPC-A)**
            The 12 digits of a UPC code are broken into three main components:
            *   **Number system character (1 digit):** The first digit is a number system character that indicates how the code should be classified.
                *   `0`, `6`, `7`, `8`, `9`: Standard UPC for most consumer products.
                *   `2`: For variable-weight items like produce or meat.
                *   `3`: For pharmaceuticals and health products.
                *   `4`: For in-store use or special applications.
                *   `5`: For coupons.
            *   **Company prefix (6–10 digits):** The first six to ten digits are the company prefix, which is assigned by GS1 and uniquely identifies the product's brand owner.
            *   **Item reference number (1–5 digits):** The item number is assigned by the brand owner to identify a specific product. The number of digits varies depending on the length of the company prefix. For example, a 16-gigabyte phone would have a different item number than a 32-gigabyte model.
            *   **Check digit (1 digit):** The final digit is a check digit, calculated using a formula involving all the other numbers in the sequence. This digit helps ensure the integrity of the code during scanning.
        *   **Other UPC formats**
            While UPC-A is the most common format, other variants exist:
            *   **UPC-E:** A shorter, 6-digit version used for smaller retail items, like cosmetics, that don't have enough space for the standard UPC-A barcode.
    *   **`Price`**: Locate the price on the electronic shelf label (ESL) positioned directly beneath the product.
    *   **`Description`**: Capture the main product title and any key variations or features mentioned on the packaging.
    *   **`Size`**: Extract these from the packaging (e.g., '12 OZ' or '1.0 kg'). The `Size` should be the numerical value.
    *   ** `UnitOfMeasure`:** should be the unit string (e.g., 'lb','ct','oz','Gallon', or 'pint'). If multiple units are present, prefer the one most commonly used for that product type.
4.  **Strictness:** Do not infer, guess, or make up any values. If you cannot clearly and confidently determine the value for a field, omit that field from the final JSON object.
5.  **ProcessingStatus Field (MANDATORY):** This field is required in every product object.
    *   **Success:** If you were able to confidently extract the `Brand` and `Price`, set the value to: `"SUCCESS"`.
    *   **Error/Clarification:** If you encounter significant issues—such as an unreadable image, ambiguous text, or the inability to confidently extract the key information—set the value to an informative error message. Use one of the following recommended structures, or provide a custom message specifying the issue:
        *   `"ERROR: IMAGE UNCLEAR - Cannot read price."`
        *   `"ERROR: DATA AMBIGUOUS - Multiple products visible, unclear which is primary."`
        *   `"CLARIFICATION REQUIRED - Please provide a clearer photo or specify the exact product name."`

---
**GOAL:** Produce a single, valid JSON array of objects that adheres to the schema and extraction rules based on the provided image.

You **MUST** adhere strictly to the following JSON Schema for your output. Do not include any text, explanations, or dialogue outside of the JSON object.

**JSON Schema:**
```json

  "$schema": "http://json-schema.org/draft-07/schema#",
  "$id": "http://example.com/products.schema.json",
  "title": "Products",
  "description": "A collection of products.",
  "type": "array",
  "items": 
    "title": "Product (Tolerant Schema)",
    "description": "A product object where all fields are optional for robustness, and includes a status field for LLM processing feedback.",
    "type": "object",
    "properties": 
      "UPC": 
        "type": "string",
        "description": "The UPC of the product."
      }},
      "Brand": 
        "type": "string",
        "description": "The brand name of the product."
      }},
      "Description": 
        "type": "string",
        "description": "A brief description of the product."
      }},
      "Price": 
        "type": "number",
        "format": "float",
        "description": "The price of the product.",
        "minimum": 0
      }},
      "Size": 
        "type": "number",
        "description": "The size of the product."
      }},
      "UnitOfMeasure": 
        "type": "string",
        "description": "The unit of measure for the quantity (e.g., L, oz, ct)."
      }},
      "ProcessingStatus": 
        "type": "string",
        "description": "Status of the extraction process. Should be 'SUCCESS' if all clear, or an error message/instruction for clarification if a problem (e.g., blurry image, ambiguous text) was encountered."
      }}
    }},
    "required": [
      "ProcessingStatus"
    ],
    "additionalProperties": false
  }}
}}
```

"""

In [ ]:
response_schema = {
  "type": "OBJECT",
  "properties": {
    "product_count": {
      "type": "INTEGER",
      "description": "The total number of products extracted."
    },
    "products": {
      "type": "ARRAY",
      "description": "A list of extracted products.",
      "items": {
        "type": "OBJECT",
        "properties": {
          "UPC": {
            "type": "STRING",
            "description": "The UPC of the product."
          },
          "Brand": {
            "type": "STRING",
            "description": "The brand name of the product."
          },
          "Description": {
            "type": "STRING",
            "description": "A brief description of the product."
          },
          "Price": {
            "type": "NUMBER",
            "description": "The price of the product."
          },
          "Size": {
            "type": "NUMBER",
            "description": "The size of the product."
          },
          "UnitOfMeasure": {
            "type": "STRING",
            "description": "The unit of measure for the size (e.g., L, oz, g)."
          }
        }
      }
    },
    "ProcessingStatus": {
      "type": "STRING",
      "description": "Overall status of the extraction process. Should be 'SUCCESS' if all clear, or an error message if a problem was encountered with the entire request."
    }
  },
  "required": [
    "product_count",
    "products",
    "ProcessingStatus"
  ]
}

### Test Single Call with an Image File (optional)

This section demonstrates a single, synchronous call to the Gemini API to process one image. The output of this cell shows the raw JSON response from the model for a single image.

In [ ]:
from google.genai.types import Part

In [ ]:
IMAGE_FILENAME = "gs://sandbox-401718-compscan-phase1/instore-selected/280029014909_Aldi_1.45.jpg"

In [ ]:
response = client.models.generate_content(
    model="gemini-2.5-pro",
    contents=[
        prompt,
        Part.from_uri(
            file_uri=IMAGE_FILENAME,
            mime_type="image/jpeg",
        ),
    ],
    config={
        "response_mime_type": "application/json",
        "response_schema": response_schema,
        "system_instruction": system_instructions,
    },

)
print(response.text)

## Analysis of batch of in-store images (Scalable / Auotmated)

In [5]:
from google.cloud import storage
import asyncio
import pandas as pd
import json
import re
from google.genai.types import Part

In [6]:
BUCKET_NAME = "sandbox-401718-compscan-phase1"
IMAGE_PREFIX = "instore-selected/"
CONCURRENT_REQUESTS = 5  # Maximum number of concurrent API requests

### Create a List of Image Files

This section prepares for batch processing by creating a list of image URIs from a specified Google Cloud Storage (GCS) bucket. It uses the GCS client to list all the image files in the bucket and stores their URIs in a list. This list will be used in the next section to make asynchronous calls to the Gemini API, allowing for the efficient processing of multiple images.

In [7]:
# Initialize the GCS client
storage_client = storage.Client()

print(f"Listing files from 'gs://{BUCKET_NAME}' with prefix '{IMAGE_PREFIX}'...")
image_uris = []

# List all blobs (files) in the specified bucket and prefix
blobs = storage_client.list_blobs(BUCKET_NAME, prefix=IMAGE_PREFIX)

for blob in blobs:
    # Construct the full gs:// URI for the file.
    image_uri = f"gs://{BUCKET_NAME}/{blob.name}"

    # Filter for common image file extensions
    if blob.name.lower().endswith(('.jpg', '.jpeg', '.png')):
        # Ensure we don't add the folder itself if it appears in the list
        if not blob.name == IMAGE_PREFIX:
            image_uris.append(image_uri)

# uncomment the line below to process subset for testing
# image_uris = image_uris[:5]

print(f"{len(image_uris)} images to process.")
# print(image_uris)

Listing files from 'gs://sandbox-401718-compscan-phase1' with prefix 'instore-selected/'...
15 images to process.


### Gemini Outputs (Async)

This section defines a function to call the Gemini API asynchronously. It then creates a list of tasks for all image URIs and runs them concurrently, which significantly speeds up the process of analyzing a large number of images compared to a sequential loop.


In [8]:
# Synchronous Worker Function
def generate_content_sync(image_uri):
    """
    This is the SYNCHRONOUS function that contains the exact API call
    that you have proven works. It will be run concurrently.
    """
    try:
        contents = [
            prompt,
            Part.from_uri(file_uri=image_uri, mime_type="image/jpeg"),
        ]
        config = {
            "response_mime_type": "application/json",
            # "response_schema": response_schema,
            "system_instruction": system_instructions,
        }

        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=contents,
            config=config,
        )
        
        print(f"Completed: {image_uri.split('/')[-1]}")
        return response.text
    
    except Exception as e:
        print(f"Failed:    {image_uri.split('/')[-1]} | Error: {e}")
        error_json = {
            "product_count": 0,
            "products": [],
            "ProcessingStatus": f"ERROR: API call failed - {str(e)}"
        }
        return json.dumps(error_json)

# --- Asynchronous Wrapper for Concurrency Control ---
# This async function's job is to manage the semaphore.
async def process_image_with_semaphore(image_uri, semaphore):
    """
    Waits for the semaphore to be available, then runs the synchronous
    API call in a separate thread.
    """
    async with semaphore:
        # Once the semaphore is acquired, run the blocking function
        # using asyncio.to_thread.
        result = await asyncio.to_thread(generate_content_sync, image_uri)
        return result

# --- Orchestration Logic ---
all_results = {}
tasks = []

# A Semaphore limits how many tasks can run at the same time.
semaphore = asyncio.Semaphore(CONCURRENT_REQUESTS)

print(f"Starting async processing for {len(image_uris)} images with a concurrency of {CONCURRENT_REQUESTS}...")

# Create a list of tasks, one for each image.
# Each task is our new semaphore-aware async wrapper function.
for image_uri in image_uris:
    task = asyncio.create_task(process_image_with_semaphore(image_uri, semaphore))
    tasks.append(task)

# asyncio.gather runs all the tasks concurrently and collects their results.
results_list = await asyncio.gather(*tasks, return_exceptions=True)

# Process the results, mapping them back to their original URIs
for uri, result in zip(image_uris, results_list):
    if isinstance(result, Exception):
        print(f"A task failed for {uri} with an unexpected error: {result}")
        all_results[uri] = json.dumps({
            "product_count": 0, "products": [], "ProcessingStatus": f"ERROR: Task execution failed - {str(result)}"
        })
    else:
        all_results[uri] = result

print("...Async processing complete.")

Starting async processing for 15 images with a concurrency of 5...
Completed: 280029014909_Aldi_1.45.jpg
Completed: 209815404397_Aldi_0.15.jpg
Completed: 253097005421_Aldi_1.09.jpg
Completed: 253864508889_Aldi_7.59.jpg
Completed: 28400516464_Aldi_4.79.jpg
Completed: 12804_Aldi_0.89.jpg
Completed: 4099100110005_Aldi_4.69.jpg
Completed: 4099200138800_Aldi_3.79.jpg
Completed: 4050_Aldi_2.78.jpg
Completed: 55124142358_Aldi_4.85.jpg
Completed: 676706875_Aldi_0.95.jpg
Completed: 595544334785_Aldi_1.99.jpg
Completed: 7751262002907_Aldi_2.59.jpg
Completed: 91453564027_Aldi_1.39.jpg
Completed: 715756200115_Aldi_3.89.jpg
...Async processing complete.


### Format Outputs and Export

This section takes the JSON outputs from the asynchronous Gemini API calls and formats them into a pandas DataFrame.

In [9]:
### Format and Export to JSON to be used in Analytics Notebook

processed_llm_output = {}

for gcs_uri, json_string in all_results.items():
    # Extract the filename from the GCS URI

    filename = os.path.basename(gcs_uri)
    try:
        data = json.loads(json_string)

        # Check if the parsed data is a list (the dict_A format)

        if isinstance(data, list):
            top_level_status = "SUCCESS"  # Assume success by default
            for product in data:
                # Check if the product is a dict and has a status

                if isinstance(product, dict) and "ProcessingStatus" in product:
                    if product["ProcessingStatus"] != "SUCCESS":
                        top_level_status = product["ProcessingStatus"]
                        break  # Found an error/warning, no need to look further
            normalized_data = {
                "product_count": len(data),
                "products": data,  # The original list becomes the value for the 'products' key
                "ProcessingStatus": top_level_status,
            }

            # Store the newly structured data
            processed_llm_output[filename] = normalized_data
            
        elif isinstance(data, dict):
            processed_llm_output[filename] = data
        else:
            processed_llm_output[filename] = {
                "error": f"Unexpected data type found: {type(data).__name__}"
            }

    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from LLM output for {filename}: {e}")
        # Store the error so you know the LLM produced invalid JSON

        processed_llm_output[filename] = {
            "error": f"JSON Decode Error: {e}",
            "product_count": 0,
            "products": [],
            "ProcessingStatus": "ERROR: Invalid JSON",
        }
        
# Save File
with open("instore_image_predict.json", "w", encoding="utf-8") as f:
    json.dump(processed_llm_output, f, indent=4)
print("Processing complete. File 'instore_image_predict.json' has been saved.")

Processing complete. File 'instore_image_predict.json' has been saved.


In [17]:
## Dataframe

dataframe_rows = []

for image_uri, response_string in all_results.items():
    try:
        parsed_data = json.loads(response_string)

        product_list = None
        processing_status = "UNKNOWN"  # Default status

        # Check if the parsed data is a dictionary or a list
        if isinstance(parsed_data, dict):
            product_list = parsed_data.get("products")
            processing_status = parsed_data.get(
                "ProcessingStatus", "SUCCESS"
            ) 
        elif isinstance(parsed_data, list):
            
            # The parsed data IS the product list
            product_list = parsed_data
            processing_status = "SUCCESS"  # Assume success if we got a list of products
        else:
            
            # Handle cases where JSON is valid but not a dict or list (e.g., "null", "true", 123)
            print(
                f"Warning: Parsed JSON for {image_uri} is not a dict or list. Type: {type(parsed_data)}"
            )
            dataframe_rows.append(
                {
                    "gcs_uri": image_uri,
                    "ProcessingStatus": "ERROR: UNEXPECTED_JSON_TYPE",
                }
            )
            continue
            
        # process the product_list we extracted
        if product_list is None:
            print(f"Info: No 'products' list found for {image_uri}.")
            dataframe_rows.append(
                {"gcs_uri": image_uri, "ProcessingStatus": processing_status}
            )
        elif not product_list:
            # This handles an empty list '[]'

            print(f"Info: Empty 'products' list for {image_uri}.")
            dataframe_rows.append(
                {"gcs_uri": image_uri, "ProcessingStatus": processing_status}
            )
        else:
            
            # Process each product item in the list
            for product_item in product_list:
                row = {"gcs_uri": image_uri, "ProcessingStatus": processing_status}
                
                # Ensure product_item is a dictionary before updating
                if isinstance(product_item, dict):
                    row.update(product_item)
                else:
                    print(
                        f"Warning: Item in product list for {image_uri} is not a dictionary. Item: {product_item}"
                    )
                    row["Description"] = "Malformed product item"
                dataframe_rows.append(row)
    except json.JSONDecodeError:
        print(
            f"Warning: Failed to decode JSON for {image_uri}. Response was: {response_string}"
        )
        dataframe_rows.append(
            {"gcs_uri": image_uri, "ProcessingStatus": "ERROR: INVALID_JSON"}
        )
    except Exception as e:
        print(f"An unexpected error occurred while processing {image_uri}: {e}")
        dataframe_rows.append(
            {"gcs_uri": image_uri, "ProcessingStatus": "ERROR: UNEXPECTED_PYTHON_ERROR"}
        )
        
# Create and clean the DataFrame
df_products = pd.DataFrame(dataframe_rows)

if not df_products.empty:
    if "gcs_uri" in df_products.columns:
        df_products["filename"] = df_products["gcs_uri"].str.split("/").str[-1]
    desired_columns = [
        "filename",
        "gcs_uri",
        "Brand",
        "Description",
        "Price",
        "Size",
        "UnitOfMeasure",
        "UPC",
        "ProcessingStatus",
    ]

    final_columns = [col for col in desired_columns if col in df_products.columns]
    df_products = df_products.reindex(
        columns=final_columns
    )  # Use reindex to add missing desired columns as NaN
print("\nGenerated DataFrame (one row per extracted product):")
try:
    display(df_products)
except NameError:
    print(df_products.to_string())



Generated DataFrame (one row per extracted product):


,filename,gcs_uri,Brand,Description,Price,Size,UnitOfMeasure,UPC,ProcessingStatus
0,12804_Aldi_0.89.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,NaN,Papaya,0.89,NaN,NaN,NaN,ERROR: BRAND NOT FOUND - Brand information not...
1,209815404397_Aldi_0.15.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,NaN,Fresh Family Pack Chicken Thighs,1.55,NaN,lb,NaN,ERROR: BRAND MISSING - Cannot identify product...
2,253097005421_Aldi_1.09.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,Hannaford,Fresh All Natural Chicken Drumsticks Value Fam...,1.09,5.42,lb,NaN,SUCCESS
3,253097005421_Aldi_1.09.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,Tyson,Chicken Drumsticks,NaN,NaN,NaN,NaN,ERROR: PRICE NOT VISIBLE - Cannot extract pric...
4,253864508889_Aldi_7.59.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,NaN,Extra Lean Ground Beef 96% LEAN / 4% FAT,8.88,1.17,Lb,2153664750888,SUCCESS
5,280029014909_Aldi_1.45.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,Simply Nature,Organic Zucchini,1.45,NaN,NaN,NaN,SUCCESS
6,28400516464_Aldi_4.79.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,Doritos,Nacho Cheese Flavored Tortilla Chips,NaN,NaN,NaN,NaN,"ERROR: IMAGE UNCLEAR - Cannot read price, UPC,..."
7,4050_Aldi_2.78.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,NaN,Loose Cantaloupe Melons,2.67,NaN,NaN,NaN,SUCCESS
8,4099100110005_Aldi_4.69.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,APPLETON FARMS,Center Cut Bacon,4.69,12.00,oz,NaN,SUCCESS
9,4099200138800_Aldi_3.79.jpg,gs://sandbox-401718-compscan-phase1/instore-se...,Pur Aqua,"Spring Water, 24 16.9 oz Bottles",NaN,24.00,ct,NaN,ERROR: PRICE MISSING - No electronic shelf lab...


In [ ]:
# Export to JSON & CSV
df_products.to_csv('products_export.csv', index=False)

### REST Example - Single Call (optional)

This section provides an alternative method for calling the Gemini API using a raw REST request. This approach is useful for environments where the Python SDK is not available or for testing the API at a lower level. The code demonstrates how to authenticate, construct the JSON payload, and make a POST request to the Gemini API endpoint.

In [ ]:
import base64
from google.cloud import storage
import json
import google.auth.transport.requests
import google.auth
import requests

In [ ]:
MODEL_ID = "gemini-2.5-pro"
gcs_image = "gs://sandbox-401718-compscan-phase1/wba/PXL_20251021_191152605.jpg"

In [ ]:
def convert_to_base_64(gcs_uri: str) -> str:
    """
    Downloads an image from a GCS URI and converts it to a Base64 string.
    
    This is a pure Python solution using the google-cloud-storage library.
    """
    storage_client = storage.Client()
    
    # Parse the GCS URI to get the bucket and file name
    bucket_name, blob_name = gcs_uri.replace("gs://", "").split("/", 1)
    blob = storage_client.bucket(bucket_name).blob(blob_name)
    
    # Download the file's content as bytes
    image_bytes = blob.download_as_bytes()
    return base64.b64encode(image_bytes).decode('utf-8')

base_64 = convert_to_base_64(gcs_image)

print("Base64 conversion complete.")
print("Preview:", output[:100] + "...")

In [149]:
request_payload = {
    "contents": [
        {
            "role": "user",
            "parts": [
                {
                    "text": "\n\nAnalyze the provided image of a grocery item or shelf.\n\nYou are a highly reliable JSON extraction engine. Your task is to analyze the provided image of a product shelf display and extract structured data for **all identifiable products**.\n\nYou **MUST** adhere strictly to the JSON output format shown in the example below. Do not include any text, explanations, or dialogue outside of the single, valid JSON object.\n\n### Extraction Rules:\n\n1. **Extractable Fields:** For each product, focus on extracting the following properties: Name, Brand, Manufacturer, Description, Price, and UPC.\n2. **Strictness and \"NONE\" Value:** Do not infer, guess, or make up any values. If you cannot clearly and confidently determine the value for a specific field (e.g., the text is blurry, price is obscured, UPC is not visible), you **MUST** use the string value `\"NONE\"` for that field. Do not omit the key.\n3. **ProcessingStatus Field (MANDATORY):** This top-level field is required.\n  *  **Error/Clarification:** If the entire image is unreadable or too ambiguous to extract any items, set the value to an informative error message (e.g., \"ERROR: IMAGE UNCLEAR\", \"ERROR: NO PRODUCTS VISIBLE\").\n\n---\n**GOAL:** Produce a single, valid JSON object containing all identified products, adhering strictly to the rules and format below.\n\n**Example of the EXACT desired output format:**\n```json\n{\n \"product_count\": 2,\n \"products\": [\n  {\n   \"Name\": \"12-Pack Classic Soda\",\n   \"Brand\": \"Coke-A-Cola\",\n   \"Manufacturer\": \"The Coke-A-Cola Company\",\n   \"Description\": \"A 12-pack of 12 fl oz classic soda cans.\",\n   \"Price\": 7.99,\n   \"UPC\": \"049000028904\"\n  },\n  {\n   \"Name\": \"Original Potato Chips\",\n   \"Brand\": \"Chip Co.\",\n   \"Manufacturer\": \"NONE\",\n   \"Description\": \"Family Size Original Potato Chips.\",\n   \"Price\": 4.50,\n   \"UPC\": \"NONE\"\n  }\n ],\n \"ProcessingStatus\": \"SUCCESS\"\n}\n"
                },
                {
                    "inline_data": {
                        "mime_type": "image/jpeg",
                        "data": base_64
                    }
                }
            ]
        }
    ],
    "systemInstruction": {
        "parts": [
          {
            "text": system_instructions
          }
        ]
      },
    "generationConfig": {
        "responseMimeType": "application/json",
        "responseSchema": {
            "type": "OBJECT",
            "properties": {
                "ProcessingStatus": { "type": "STRING" },
                "product_count": { "type": "INTEGER" },
                "products": {
                    "type": "ARRAY",
                    "items": {
                        "type": "OBJECT",
                        "properties": {
                            "UPC": { "type": "STRING" },
                            "Name": { "type": "STRING" },
                            "Description": { "type": "STRING" },
                            "Manufacturer": { "type": "STRING" },
                            "Price": { "type": "NUMBER" },
                            "Brand": { "type": "STRING" }
                        },
                        "required": ["Name", "Brand", "Manufacturer", "Description", "Price", "UPC"]
                    }
                }
            },
            "required": ["product_count", "products", "ProcessingStatus"]
        }
    },
    "model": "projects/sandbox-401718/locations/us-central1/publishers/google/models/gemini-2.5-pro"
}

# Convert the Python dictionary to a JSON string
request_body = json.dumps(request_payload, indent=2)

In [151]:
# Credentials

# Set up Application Default Credentials (ADC)
credentials, project_id = google.auth.default()
auth_req = google.auth.transport.requests.Request()
credentials.refresh(auth_req)
access_token = credentials.token

In [152]:
api_host = f"{LOCATION}-aiplatform.googleapis.com"

url = f"https://{api_host}/v1/projects/{PROJECT_ID}/locations/{LOCATION}/publishers/google/models/{MODEL_ID}:generateContent"

headers = {
    'Authorization': 'Bearer ' + access_token,
    'Content-Type': 'application/json; charset=utf-8'
}

In [153]:
response = requests.post(url, data=request_body, headers=headers)
print(response.status_code)

200


In [154]:
# Then, get the JSON data from the response
response_data = response.json()

# Access the text content
raw_text_content = response_data['candidates'][0]['content']['parts'][0]['text']

# The 'text' field itself contains a JSON string, so we need to parse it again
final_content_output = json.loads(raw_text_content)

# You can print it directly or pretty print it
print(json.dumps(final_content_output, indent=2))

{
  "product_count": 2,
  "products": [
    {
      "Name": "Frosted Flakes",
      "Brand": "Kellogg's",
      "Manufacturer": "Kellogg's",
      "Description": "Large Size, of corn. Net Wt 17.3 oz (1 LB 1.3 OZ) (490g).",
      "Price": 7.29,
      "UPC": "03800028190"
    },
    {
      "Name": "Honey Smacks",
      "Brand": "Kellogg's",
      "Manufacturer": "Kellogg's",
      "Description": "Large Size, Sweetened Puffed Wheat Cereal, 15.3 oz.",
      "Price": 7.29,
      "UPC": "03800039103"
    }
  ],
  "ProcessingStatus": "SUCCESS"
}
